In [1]:
import numpy as np

In [2]:

from PIL import Image
import pytesseract
import argparse
import cv2
import os
 


In [3]:
URL = "/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/OCR部分/7F2ACB09-422F-4214-BA20-C8A04F4D727E.png"

In [4]:
import pytesseract
from PIL import Image


im_en = Image.open(URL)

data = [i for i in pytesseract.image_to_string(im_en).splitlines() if len(i) > 0 and i != '/']
data

['iBMiZiX iE: farok crrrok hihok yorok clok kantok ok-yurp',
 'la. ok-voon ororok sprok .',
 '1b. at-voon bichat dat .',
 '2a. ok-drubel ok-voon anok plok sprok .',
 '2b. at-drubel at-voon pippat rrat dat .',
 '3a. erok sprok izok hihok ghirok .',
 '3b. totat dat arrat vat hilat .',
 '4a. ok-voon anok drok brok jok .',
 '4b. at-voon krat pippat sat lat .',
 'Sa. wiwok farok izok stok .',
 '5b. totat jjat quat cat .',
 '6a. lalok sprok izok jok stok .',
 '6b. wat dat krat quat cat .',
 'Ta. lalok farok ororok lalok sprok izok enemok .',
 '7b. wat jjat bichat wat dat vat eneat .',
 '8a. lalok brok anok plok nok .',
 '8b. iat lat pippat rrat npat .',
 '9a. wiwok nok izok “pms -',
 '9b. totat nnat quat oloat at-yurp .',
 '10a. lalok mok nok yorok ghirok clok .',
 '10b. wat nnat gat mat bat hilat .',
 '11a. lalok nok crrrok hihok yorok zanzanok .',
 '11b. wat nnat arrat mat zanzanat .',
 '12a. lalok rarok nok izok hihok mok .',
 '12b. wat nnat forat arrat vat gat .']

# 原文

In [5]:
original = np.array(data[0].split(': ')[-1].split(' '))
original

array(['farok', 'crrrok', 'hihok', 'yorok', 'clok', 'kantok', 'ok-yurp'],
      dtype='<U7')

# 语料

In [6]:

def Split(tokenization,n=2):
    math = __import__("math")
    return [tokenization[i*n:i*n+n] for i in range(len(tokenization)//2)]

In [7]:
corpus_list = [i.split(".")[1] for i in data[1:]]
corpus = Split(corpus_list)
corpus

[[' ok-voon ororok sprok ', ' at-voon bichat dat '],
 [' ok-drubel ok-voon anok plok sprok ',
  ' at-drubel at-voon pippat rrat dat '],
 [' erok sprok izok hihok ghirok ', ' totat dat arrat vat hilat '],
 [' ok-voon anok drok brok jok ', ' at-voon krat pippat sat lat '],
 [' wiwok farok izok stok ', ' totat jjat quat cat '],
 [' lalok sprok izok jok stok ', ' wat dat krat quat cat '],
 [' lalok farok ororok lalok sprok izok enemok ',
  ' wat jjat bichat wat dat vat eneat '],
 [' lalok brok anok plok nok ', ' iat lat pippat rrat npat '],
 [' wiwok nok izok “pms -', ' totat nnat quat oloat at-yurp '],
 [' lalok mok nok yorok ghirok clok ', ' wat nnat gat mat bat hilat '],
 [' lalok nok crrrok hihok yorok zanzanok ', ' wat nnat arrat mat zanzanat '],
 [' lalok rarok nok izok hihok mok ', ' wat nnat forat arrat vat gat ']]

# 统计翻译系统转译

In [8]:
def set_word(sentence):
    return np.array(list(filter(lambda x : x != '' , sentence.split(' '))))

In [9]:
def statistical(corpus):
    n = -1
    tn = 0
    i_n = -1
    result_t,result_w = [],[]
    for sentence in corpus:
        n += 1
        word_left , word_right = set_word(sentence[0]),set_word(sentence[-1])
        temp1 = np.intersect1d(word_left,original) 
        if temp1.size > 0:
        
            for t in temp1:
                t,w = t,word_right[np.argwhere(word_left==t)[0][0]]
                if t in result_t:
                    tn += 1
                    index_number = result_t.index(t)
                    result_w.insert(index_number,(index_number,tn,w))
                else:
                    i_n += 1
                    tn = 0
                    result_t.append(t)
                    result_w.append((i_n,tn,w))
    return result_t,sorted(result_w)

In [10]:
statistical(corpus)

(['hihok', 'farok', 'clok', 'yorok', 'crrrok'],
 [(0, 0, 'vat'),
  (0, 1, 'mat'),
  (0, 3, 'vat'),
  (1, 0, 'jjat'),
  (1, 1, 'jjat'),
  (2, 0, 'hilat'),
  (3, 0, 'mat'),
  (3, 2, 'zanzanat'),
  (4, 0, 'arrat')])

In [11]:
def translation(original):
    result_t,result_w = statistical(corpus)
    encode_dict = dict()
    temp_number = 0
    for t in result_t:
        temp_number += 1
        temp_index_str = "{}{}".format(temp_number,t)
        temp_name_str = "{}{}{}".format(temp_number,t,t)
        temp_index,temp_name = [],[]
        for w in result_w:
            if result_t[w[0]]==t:
                temp_index.append(w[1])
                temp_name.append(w[-1])
        max_index = temp_index.index(max(temp_index))
        temp_dict = {t:(temp_name[max_index],temp_name)}
        encode_dict.update(temp_dict)
    result = {i:i in result_t and encode_dict[i] or 'error' for i in original }
    return {"encode dict":encode_dict,"translation":result}

In [12]:
translation(original)

{'encode dict': {'hihok': ('vat', ['vat', 'mat', 'vat']),
  'farok': ('jjat', ['jjat', 'jjat']),
  'clok': ('hilat', ['hilat']),
  'yorok': ('zanzanat', ['mat', 'zanzanat']),
  'crrrok': ('arrat', ['arrat'])},
 'translation': {'farok': ('jjat', ['jjat', 'jjat']),
  'crrrok': ('arrat', ['arrat']),
  'hihok': ('vat', ['vat', 'mat', 'vat']),
  'yorok': ('zanzanat', ['mat', 'zanzanat']),
  'clok': ('hilat', ['hilat']),
  'kantok': 'error',
  'ok-yurp': 'error'}}

# 翻译结果

In [13]:
translation(original)['translation']

{'farok': ('jjat', ['jjat', 'jjat']),
 'crrrok': ('arrat', ['arrat']),
 'hihok': ('vat', ['vat', 'mat', 'vat']),
 'yorok': ('zanzanat', ['mat', 'zanzanat']),
 'clok': ('hilat', ['hilat']),
 'kantok': 'error',
 'ok-yurp': 'error'}

# 缺点：
## 1 慢，
## 2 无法考虑语境，
## 3 还有语法也是不对的，
## 4 因为是按照单词对应顺序，
## 5 需要大量的规则统计，
## 6 所以没有实用性

### 今晚 ｜ 的 ｜ 课程 ｜ 有意思 <-- Broken English --> tonight｜ of ｜ the course ｜ interesting
<img src="/Users/manmanzhang/Library/Mobile Documents/com~apple~CloudDocs/MyProject/InferenceSystem/src/I5_algorithm/NLP训练营笔记/图片/52DF7BD1-F1B2-49A7-ABDF-EFB26446FAAB.png">

In [15]:
chinese = "今晚","的","课程","有意思"
english = "tonight","of","the course","interesting"